In [1]:
from os import mkdir
from os.path import exists
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.model_selection import RandomizedSearchCV, ParameterSampler, GridSearchCV
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sqlalchemy import create_engine
import sqlalchemy
import pyodbc
import urllib
import warnings
warnings.filterwarnings('ignore')
import pickle
from tqdm import tqdm
from tools import get_connection, entitle, fill, permutation_importances, scores
import tree_explainer
from datetime import datetime
from scipy.stats import geom, uniform, randint
from scipy.spatial import ConvexHull
from scipy.interpolate import UnivariateSpline
from subprocess import call

#For Space issue
import os
os.environ['JOBLIB_TEMP_FOLDER'] = '/tmp'

/opt/conda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Location of models etc.
folder = '/home/jovyan/Tech_Index/general_models/'

# Location to output CSV files
outfolder = folder

conn_str='Driver={ODBC Driver 17 for SQL Server};Server=uschwsql1056d;Database=DEV_ANALYTICS;Trusted_Connection=yes;'

# Set up a SQL conection
engine = get_connection()

#months to consider for training dataset
m_cons = 12

# Number of new hyperparameter combinations to check for each model:
gridsize = 1    # just testing; will change later

# Maximum number of data points allowed in a country's 
# training data (for the sake of memory and speed)
maxsize = 500000

In [3]:
# A function for converting SQL feature names into plain English...
def feature_english(f):
    f = f.replace('SICMAJORGRPCD', 'vertical number (tricky variable)')
    f = f.replace('SMBSHARE', 'proportion of resellers classified SMB')
    f = f.replace('TRANSCOUNT', 'number of transactions X')
    f = f.replace('PRODUCTCOUNT', 'number of distinct products transacted X')
    f = f.replace('BCNCOUNT', 'number of distinct resellers dealt with X')
    f = f.replace('EXTENDEDSALES', 'revenue X')
    f = f.replace('DAYSSINCELASTTRANS', 'time passed since the last transaction')
    f = f.replace('EmployeesTotalFinal', 'seat size')
    f = f.replace('QMONTH', 'month within the quarter')
    f = f.replace('MONTH', 'month of the year')
    f = f.replace('QUARTER', 'quarter of the year')
    f = f.replace('X1M', 'in the past month')
    f = f.replace('X3M', 'in the past 3 months')
    f = f.replace('X6M', 'in the past 6 months')
    f = f.replace('X2Q', '2 quarters ago')
    f = f.replace('X3Q', '3 quarters ago')
    f = f.replace('X4Q', '4 quarters ago')   # should
    f = f.replace('X5Q', '5 quarters ago')   # have
    f = f.replace('X6Q', '6 quarters ago')   # used
    f = f.replace('X7Q', '7 quarters ago')   # regex
    f = f.replace('X8Q', '8 quarters ago')
    f = f.replace('X', 'in the past year')
    return f

In [4]:
conn = pyodbc.connect(conn_str)

# Get the CALMONTHID of the most recent labeled predictions
calmon = pd.read_sql(
    '''
    select max(CALMONTHID)
    from DEV_ANALYTICS.dbo.TECHINDEX
    where [LEVEL] = 'GENERAL'
    and LABEL is not NULL
    ''',
    conn
).iloc[0,0]



# Get a list of all COMPANYCD
ccd = pd.read_sql(
    '''
    select COMPANYCD
    from DEV_ANALYTICS.DBO.TECHINDEX   
    where [LEVEL] = 'GENERAL' and CALMONTHID = %d
    group by COMPANYCD
    order by COMPANYCD
    ''' % calmon,
    conn
)
conn.close()





# Graph size parameters
plt.rcParams['figure.figsize'] = 10, 15
plt.rcParams['font.size'] = 12





entitle("Measuring last month's performance and building new standardizing functions...")

# Loop over all COMPANYCD
for i in tqdm(range(len(ccd))):

    COMPANYCD = ccd.iloc[i].COMPANYCD
    
    conn = pyodbc.connect(conn_str)
    # Get last month's outcomes
    results = pd.read_sql(
        "SELECT RAWSCORE as pred, label as truth " +
        "FROM DEV_ANALYTICS.dbo.TECHINDEX " +
        "where companycd = '" + COMPANYCD +
        "' and [LEVEL] = 'GENERAL" +
        "' and CALMONTHID = " + str(calmon) +
        ";", 
        conn
    )
    conn.close()

    # Skip if COMPANYCD doesn't have both positives and negatives
    if len(set(results.truth)) < 2: 
        continue

        
    ################################################
    #      FIRST PLOT (UPPER LEFT): AUC CURVE      #
    ################################################
    plt.subplot(3,2,1)

    fpr, tpr, _ = metrics.roc_curve(results.truth, results.pred)

    plt.plot([0, 1], [0, 1], linestyle='--', c = 'black', lw = .5)
    plt.plot(fpr, tpr, c='red', lw = 3)
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title(
        'ROC Curve (AUC %.4f)' % 
        (metrics.roc_auc_score(results.truth, results.pred))
    )

    
    ###########################################################
    #    SECOND PLOT (UPPER RIGHT): PRECISION-RECALL CURVE    #
    ###########################################################
    plt.subplot(3,2,2)

    fpr, tpr, _ = metrics.precision_recall_curve(results.truth, results.pred)
    precision = metrics.precision_score(results.truth, results.pred > .3)
    recall = metrics.recall_score(results.truth, results.pred > .3)
    plt.plot(fpr, tpr, c='red', lw = 3, zorder = 10)
    plt.grid()
    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.xlabel('Precision')
    plt.ylabel('Recall')
    plt.title('Precision & Recall')


    #######################################################
    #        THIRD PLOT (MIDDLE LEFT): GAIN CHART         #
    #######################################################
    plt.subplot(3,2,3)

    results = results.sort_values('pred', ascending = False)
    results['rand'] = results.sample(frac = 1).truth.values
    results['wiz'] = results.sort_values('truth', ascending = False).truth.values
    x = np.linspace(0, 1, 51 if len(results) >= 500 else 21)
    
    y_t = results.truth.sum()
    y_r = []
    y_m = []
    y_w = []
    for p in x:
        y_r.append(results.head(int(len(results)*p)).rand.sum()/y_t)
        y_m.append(results.head(int(len(results)*p)).truth.sum()/y_t)
        y_w.append(results.head(int(len(results)*p)).wiz.sum()/y_t)

    y_r, y_m, y_w = fill(y_r), fill(y_m), fill(y_w)
        
    plt.plot(x,y_r, lw = 3)
    plt.plot(x,y_m, lw = 3)
    plt.plot(x,y_w, lw = 3)
    plt.xlabel('% from top')
    plt.ylabel('% of all positives')
    vals = plt.gca().get_xticks()
    plt.gca().set_xticklabels(['{:3.0f}%'.format(x*100) for x in vals])
    vals = plt.gca().get_yticks()
    plt.gca().set_yticklabels(['{:3.0f}%'.format(x*100) for x in vals])
    plt.title('Cumulative Gain')
    plt.legend(['Random', 'Model', 'Perfect'])


    ######################################################
    #       FOURTH PLOT (MIDDLE RIGHT): K-S CHART        #
    ######################################################
    plt.subplot(3,2,4)

    y_f = (results.truth == 0).sum()
    y_n = []
    for p in x:
        y_n.append((results.head(int(len(results)*p)).truth == 0).sum()/y_f)

    y_n = fill(y_n)

    KS = [y_m[i] - y_n[i] for i in range(len(x))]
    KSi = KS.index(max(KS))

    plt.plot(x,y_m, c='blue', lw = 3, zorder = 10)
    plt.plot(x,y_n, c='red', lw = 3, zorder = 10)
    plt.plot([x[KSi],x[KSi]], [y_n[KSi], y_m[KSi]], c='gray', lw = 5)
    plt.text(x[KSi] + .01, (y_n[KSi] + y_m[KSi])/2, 'KS: %.2f' % max(KS), zorder = 6969)

    plt.xlabel('% from top')
    vals = plt.gca().get_xticks()
    plt.gca().set_xticklabels(['{:3.0f}%'.format(x*100) for x in vals])
    vals = plt.gca().get_yticks()
    plt.gca().set_yticklabels(['{:3.0f}%'.format(x*100) for x in vals])

    plt.title('Kolomogorov-Smirnov')
    plt.legend(['% of all positives', '% of all negatives'])


    #########################################################
    #    FIFTH PLOT (BOTTOM LEFT): CUMULATIVE LIFT CHART    #
    #       AND CREATION OF STANDARDIZING FUNCTION!!        #
    #########################################################
    plt.subplot(3,2,5)
  
    x = np.concatenate((
        np.linspace(0, x[1], 51 if len(results) >= 500 else 21), 
        x[2:]
    ))
    
    y_l = []
    for p in x:
        y_l.append(
            results.head(int(len(results)*p)).truth.mean() 
            / 
            results.truth.mean() - 1
        )

    try:
        ix = min([
            i for i in range(len(x) - 1) 
            if (
                int(len(results)*x[i]) >= 20
                or x[i] >= .02
            )
            and y_l[i] > 0
        ])
        x = x[ix:]
        y_l = y_l[ix:]    

        hull = ConvexHull([[x[i], y_l[i]] for i in range(len(x))])

        ihull = np.roll(
            hull.vertices,
            -list(hull.vertices).index(0)
        )

        ihull = ihull[:list(ihull).index(max(ihull))+1]

        xhull = [x[i] for i in ihull]
        yhull = [y_l[i] for i in ihull]

        spf = UnivariateSpline(xhull, yhull, k = 1, s = 0, ext = 'const')

        with open(folder + '/spf_' + COMPANYCD + '.pkl', 'wb') as f:
            pickle.dump(spf, f)

        xplot = np.linspace(0, 1, 1000)
        plt.plot(xplot, spf(xplot), lw = 2, c='red', zorder = 42)
        spf_done = True
        
    except:
        spf_done = False


    plt.plot(x, y_l, marker = 'o', linewidth = 0, zorder = 9001, c = 'blue')
    plt.plot(x, y_l, linewidth = 1, c = 'blue')
    plt.xlabel('% from top')
    plt.ylabel('Cumulative lift (minus 1)')
    vals = plt.gca().get_xticks()
    plt.gca().set_xticklabels(['{:3.0f}%'.format(x*100) for x in vals])
    plt.title('Cumulative lift')
    if spf_done:
        plt.legend(['Standardizer', 'Tested lift'])
        
        
    #####################################################
    #    SIXTH PLOT (BOTTOM RIGHT): LOCAL LIFT CHART    #
    #####################################################
    plt.subplot(3,2,6)

    x = np.linspace(0, 1, 21 if len(results) >= 500 else 11)
    
    y_r = []
    y_l = []
    dx = x[1] - x[0]
    for p in x[1:]:
        y_r.append(1)
        y_l.append(
            results.iloc[
                int(len(results)*(p-dx)):int(len(results)*p)
            ].truth.mean() 
            / results.rand.mean()
        )

    Qi = min([i for i in range(len(x)-1) if y_l[i] < y_r[i]]) - 1

    plt.plot(x[1:],y_r, lw = 3)
    plt.plot(x[1:],y_l, lw = 3)

    if y_l[Qi] > y_l[Qi+1]:
        Qx = x[Qi+2] 
        Qx -= (y_l[Qi+1] - 1) * (x[Qi+2] - x[Qi+1])/(y_l[Qi+1] - y_l[Qi])
    else:
        Qx = x[Qi+1]
   
    plt.scatter([Qx], [1], c='gray', zorder = 9001)
    plt.text(Qx+.01, 1.1, '{:3.0f}%'.format(Qx*100))
    vals = plt.gca().get_xticks()
    plt.gca().set_xticklabels(['{:3.0f}%'.format((1-x)*100) for x in vals])
    plt.xlabel('Percentile')
    plt.ylabel('Lift at quantile')
    plt.title('Lift')
    plt.legend(['Random', 'Model'])

    plt.suptitle('COMPANYCD ' + COMPANYCD, y = .95)
    plt.tight_layout()
    plt.subplots_adjust(top=.9)
    plt.savefig(
        folder + '/post_' + COMPANYCD + '.png', 
        facecolor = 'w', 
        transparent = False
    )
    plt.clf()





# New graph size parameters
plt.rcParams['figure.figsize'] = 10, 10
plt.rcParams['font.size'] = 14





entitle("Checking correctness of last month's TECHINDEX_VALUE...")
conn = pyodbc.connect(conn_str)
# Get last month's results 
df = pd.read_sql(
    "SELECT COMPANYCD, TECHINDEX_VALUE as impr, LABEL as truth " +
    "FROM DEV_ANALYTICS.dbo.TECHINDEX " +
    "where CALMONTHID = " + str(calmon) +
    " and [LEVEL] = 'GENERAL" + "';", 
    conn
)
conn.close()

# Compare Tech Index value (AKA promised improvement) to actual improvement
df_ = df.groupby([
    df.COMPANYCD,
    -(df.impr.apply(np.floor).astype(int))
]).agg({
    'impr'  : 'count',
    'truth' : 'sum'
})
df_.columns = ['tot', 'pos']
df_ = df_.reset_index()
df_[['tot','pos']] = df_.groupby('COMPANYCD')[['tot','pos']].transform(pd.Series.cumsum)    
df_ = df_.reset_index().join(
    df.groupby('COMPANYCD').mean().truth,
    on = 'COMPANYCD'
)
df_['Min Tech Index value'] = -(df_.impr)
df_['Actual improvement'] = (df_.pos/df_.tot)/df_.truth - 1    


# Graph the promised vs. actual improvement
xlim = int(df_['Min Tech Index value'].max()) + 1
xlim = xlim if xlim <= 20 else 20
try:
    plt.subplot(1,2,1)
    plt.scatter(
        df_['Min Tech Index value'], 
        df_['Actual improvement'], 
        alpha = .5,
        c = 'red'
    )
    plt.plot([0,xlim],[0,xlim],'k--',lw = 3)
    plt.xlim([0,xlim])
    plt.ylim([0,xlim * 2])
    plt.xlabel('Min Tech Index value') 
    plt.ylabel('Actual improvement')

    plt.subplot(1,2,2)
    for csc in df_.COMPANYCD.unique():
        plt.plot(
            df_[df_.COMPANYCD == csc]['Min Tech Index value'], 
            df_[df_.COMPANYCD == csc]['Actual improvement'], 
            lw = 2, 
        )
    plt.plot([0,xlim],[0,xlim],'k--',lw = 3)
    plt.xlim([0,xlim])
    plt.ylim([0,xlim * 2])
    plt.xlabel('Min Tech Index value') 
    plt.yticks([])
    plt.ylabel('')


    plt.tight_layout()
    plt.savefig(
        folder + '/scatter.png', 
        facecolor = 'w', 
        transparent = False
    )
except:
    pass

plt.clf()

************************************************************************************
*                                                                                  *
*  Measuring last month's performance and building new standardizing functions...  *
*                                                                                  *
************************************************************************************


100%|██████████| 26/26 [00:50<00:00,  1.96s/it]


*************************************************************
*                                                           *
*  Checking correctness of last month's TECHINDEX_VALUE...  *
*                                                           *
*************************************************************


<Figure size 720x1080 with 0 Axes>

In [5]:
with open(folder + '/last_train.pkl', 'rb') as f:
    l_calmon = pickle.load(f)

conn = pyodbc.connect(conn_str)
mon_chng = pd.read_sql(
    '''select count(distinct CALMONTHID)
    from DEV_ANALYTICS.dbo.TECHINDEX_GENERAL_FEATURESET
    where calmonthid>%s
    ''' % (l_calmon),conn
).iloc[0,0]
conn.close()

#if model hasn't been trained in last 3 months re-train it
if (mon_chng>=3):
    entitle("More than 3 months passed, Re-training models")
    conn = pyodbc.connect(conn_str)
    #Get id of last 6 calmonths training data
    t1 = pd.read_sql(
        '''select min(calmonthid) as min_calmonthid, max(calmonthid) as max_calmonthid
        from
        (select calmonthid,
        row_number() over(order by calmonthid desc) as rowid
        from DEV_ANALYTICS.DBO.TECHINDEX_GENERAL_featureset(nolock)
        group by calmonthid having count(distinct companycd)>20
        ) x1 where x1.rowid<=%s
        ''' % (m_cons), conn)
    min_calmonthid = t1['min_calmonthid'][0]
    max_calmonthid = t1['max_calmonthid'][0]

    # Get a list of COMPANYCD
    ccd = pd.read_sql(
        '''select COMPANYCD, count(*) as TOTCOUNT, sum(LABEL) as POSCOUNT
            from DEV_ANALYTICS.dbo.TECHINDEX_GENERAL_FEATURESET(nolock)
            where calmonthid>=%s
            group by COMPANYCD
            order by COMPANYCD
        ''' % (min_calmonthid), conn)
    conn.close()
    
    
    for i in range(len(ccd)):
        companycd = ccd.iloc[i].COMPANYCD

        # Skip countries with too few data points
        if ccd.iloc[i].TOTCOUNT < (100*m_cons) : continue
        if ccd.iloc[i].POSCOUNT < (10*m_cons) : continue
        #Sample run for any country
        #if companycd!='MD' : continue

        baserate = ccd.iloc[i].POSCOUNT / ccd.iloc[i].TOTCOUNT

        print("Building model for %s"  % companycd)        

        # Get the training data
        conn = pyodbc.connect(conn_str)
        df = pd.read_sql(
            '''
                select *
                from DEV_ANALYTICS.dbo.TECHINDEX_GENERAL_FEATURESET(nolock)
                where COMPANYCD = '%s' and calmonthid>=%s
                and ((Label = 1 and RowID <= %.0f)    -- pre-balancing in SQL, only
                  or (Label = 0 and RowID <= %.0f))   -- for the gigantic countries
            ''' % (
                companycd, min_calmonthid,
                maxsize/2/baserate/m_cons,
                maxsize/2/(1-baserate)/m_cons
            ),
            conn
        )
        conn.close()

        # Convert some features into numerical form
        df['SICMAJORGRPCD'] = df.SICMAJORGRPCD.astype(float)
        df['MONTH'] = df.CALMONTHID % 100 - 1
        df['QMONTH'] = df.MONTH % 3
        df['QUARTER'] = df.MONTH // 3

        # Balance the training data (if necessary and possible) 
        # by throwing away negative cases
        pos = sum(df.Label)
        negs_needed = max([pos, 1000 - pos])
        if negs_needed > len(df) - pos:
            negs_needed = len(df) - pos

        df = df[df.Label == 1].append(df[df.Label == 0].sample(n = negs_needed))    

        # Drop the columns that should never be used as features
        X = df.drop([
            'CALMONTHID', 'COMPANYCD', 'DM_ULT_DUN', 'Label', 'RowID'
        ], 1).fillna({
            'EmployeesTotalFinal' : -1,
            'SICMAJORGRPCD' : 100
        }).fillna(-9999)

        # Store the labels (bought or didn't buy) separately
        y = df['Label']

        # If there's only one kind of label, skip the country entirely
        if len(y.unique()) < 2 or sum(y) < 10: continue

        imp = permutation_importances(X, y)
        print('Found %d important features.' % sum(i_ > 1e-10 for i_ in imp))

        # Get a random selection of hyperparameter sets for optimization
        pgrid = ParameterSampler(
            {
                "estimator__max_depth"         : geom(.08, 1),
                "estimator__max_features"      : uniform(0, 1),
                "estimator__min_samples_leaf"  : np.logspace(0.2, 2.7, 100).astype(int),
                "estimator__min_samples_split" : np.logspace(.31, 3, 100).astype(int),
                "estimator__n_estimators"      : [300],
                "selector__k"                  : randint(5, X.shape[1])
            }, 
            n_iter = gridsize
        )

        # If optimal hyperparameters were previously recorded, add them back in
        try:
            with open(folder + 'params_%s.pkl' % companycd, 'rb') as f:
                oldp = pickle.load(f)
            pgrid = [p for p in pgrid] + [oldp]
        except:
            pass

        # Clean up the format of the hyperparameter sets for GridSearchCV
        pgrid = [
            {
                xx:[xv] 
                for (xx,xv) in zip(p.keys(), p.values())
            } 
            for p in pgrid
        ]     

        # Use cross-validation to find the best set of hyperparameters
        # (this is where all the time is spent)
        pipe = Pipeline([
            ('selector'  , SelectKBest(scores(imp))),
            ('estimator' , RandomForestClassifier())
        ])
        gs = GridSearchCV(
            pipe, 
            param_grid = pgrid,
            n_jobs = -2,          # Use all but one of the computer's cores
            scoring = 'roc_auc',  # Judge by AUC
    #        cv = StratifiedKFold(3, shuffle = True),
            verbose = 3           # Report progress along the way
        )
        gs.fit(X, y)
        clf = gs.best_estimator_
        print("Best cross-validation AUC: %.4f\n\n" % (gs.best_score_))

        # Save the best model and hyperparameters
        with open(folder + 'params_%s.pkl' % companycd, 'wb') as f:
            pickle.dump(gs.best_params_, f)

        with open(folder + 'clf_%s.pkl' % companycd, 'wb') as f:
            pickle.dump(clf, f)


    with open(folder + 'last_train.pkl', 'wb') as f:
        pickle.dump(max_calmonthid, f)
        
else: entitle("No need to Re-Train models")

********************************
*                              *
*  No need to Re-Train models  *
*                              *
********************************


In [6]:
conn = pyodbc.connect(conn_str)

# Get the CALMONTHID of the latest unlabeled features
calmon = pd.read_sql(
    '''select max(CALMONTHID)
    from DEV_ANALYTICS.dbo.TECHINDEX_GENERAL_FEATURESET_unlabeled
    ''',conn
).iloc[0,0]



# Get a list of all COMPANYCD's
ccd = pd.read_sql(
    '''select COMPANYCD
    from DEV_ANALYTICS.DBO.TECHINDEX_GENERAL_featureset_unlabeled   
    where CALMONTHID = %d
    group by COMPANYCD
    order by COMPANYCD
    ''' % calmon, conn
)

conn.close()

entitle("Building next month's predictions...")

******************************************
*                                        *
*  Building next month's predictions...  *
*                                        *
******************************************


In [7]:
# Before the first batch of predictions, we will generate a header
header = True

# Loop over all COMPANYCD
for i in range(len(ccd)):

    companycd = ccd.iloc[i].COMPANYCD
    start = datetime.now()

    print(companycd)

    # Try to find a classifier for this COMPANYCD
    try:
        with open(folder + '/clf_%s.pkl' % companycd, 'rb') as f:
            clf = pickle.load(f)
    except:
        continue

    # Adjust the classifier to use all CPU cores
    clf.named_steps['estimator'].set_params(n_jobs = -1)

    # Try to find a standardizing function for this COMPANYCD
    try:
        with open(folder + '/spf_%s.pkl' % companycd, 'rb') as f:
            spf = pickle.load(f)
    # If none exists, the TECHINDEX_VALUE will simply be the percentile (as a decimal)
    except:
        spf = (lambda x : 1 - x)
    
    conn = pyodbc.connect(conn_str)

    # Report the English description of the current COMPANYCD
    try:
        print(
            pd.read_sql(
                '''
                select VANITY_NAME as VN
                from BIC_REFERENCE.dbo.UI_REGION
                where COMPANYCD = '%s'
                ''' % companycd,
                conn
            ).VN[0]
        )
    except:
        print('(Country name unknown.)')

    # Get the features for this COMPANYCD
    df = pd.read_sql(
        '''
            select *
            from DEV_ANALYTICS.dbo.TECHINDEX_GENERAL_FEATURESET_unlabeled
            where COMPANYCD = '%s' and CALMONTHID = %d
        ''' % (companycd, calmon),
        conn
    )
    conn.close()
    
    # Skip this COMPANYCD if no data is found
    if len(df) < 1: 
        continue

    # Prepare the output dataframe
    ret = df.assign(
        LEVEL = 'GENERAL'
    ).assign(
        GROUP = ''
    )[['LEVEL', 'CALMONTHID', 'COMPANYCD', 'DM_ULT_DUN', 'GROUP']]

    # Convert some features into numerical form
    df['SICMAJORGRPCD'] = df.SICMAJORGRPCD.astype(float)
    df['MONTH'] = df.CALMONTHID % 100 - 1
    df['QMONTH'] = df.MONTH % 3
    df['QUARTER'] = df.MONTH // 3
    
    # Drop the columns that should never be used as features
    # and fill missing values
    X = df.drop([
        'CALMONTHID', 'COMPANYCD', 'DM_ULT_DUN', 'Label', 'RowID'
    ], 1).fillna({
        'DAYSSINCELASTTRANS' : 750,
        'EmployeesTotalFinal' : -1,
        'SICMAJORGRPCD' : 100
    }).fillna(-9999)
 
    # Report the most important features that were found when the model was trained
    print('Most important features:')
    for pair in sorted(
        zip(X.columns, clf.named_steps['selector'].score_func.imp), 
        key = lambda pair: -pair[1]
    )[:3]:
        print('*', pair[0])
    
    print('\nScoring %d end users,' % (len(df)), end = ' ')

    # Get the scores from the classifier and use them 
    # with the standardizer to produce the TECHINDEX_VALUEs
    ret['RAWSCORE'] = clf.predict_proba(X)[:,1]
    ret['TECHINDEX_VALUE'] = (
        ret.RAWSCORE.rank(axis = 0, method = 'min', ascending = False) / len(ret.RAWSCORE)
    ).apply(spf)

    # Calculate how many explanations to be produced:
    # The top 20%, up to a maximum of 10K end users
    score_threshold = ret.RAWSCORE.quantile(q = 0.8)
    rank = ret.rank(ascending = False).RAWSCORE
    index_top = (rank <= 10000) & (rank <= len(X) // 5)   

    print('with explanations for the top %d scores...' % sum(index_top), end = ' ')
    
    # Get the explanations
    if sum(index_top) > 0:
        pred_exp = tree_explainer.predict_explain(
            clf.named_steps['estimator'], 
            pd.DataFrame(
                clf.named_steps['selector'].transform(X[index_top]),
                columns = [
                    str(n) 
                    for n,b in enumerate(clf.named_steps['selector'].get_support()) 
                    if b
                ]
            ),
            confidence = 0.67
        )

    # Copy the explanations into the output dataframe
    ret['REASON1CD'] = ''
    ret['REASON2CD'] = ''
    if sum(index_top) > 0:
        ret.loc[index_top, 'REASON1CD'] = pred_exp.REASON1.values
        ret.loc[index_top, 'REASON2CD'] = pred_exp.REASON2.values    
    
    # Save the output to disk 
    # If it's the first batch, overwrite the file and use a header
    # Otherwise, append with no header
    ret.to_csv(outfolder + 'TECHINDEX.csv', header = header,
               mode = ('w' if header else 'a'), index = False)
    header = False
    
    
# Output a file of reason codes and reasons
header = True
for l, s in [('l', 'small'), ('m', 'moderate'), ('h', 'large')]:
    for i, c in enumerate(X.columns):
        with open(
            outfolder + 'TECHINDEX_REASONS.csv', 
            'w' if header else 'a'
        ) as f:
            if header: f.write('LEVEL,REASONCD,REASON\n')
            f.write('GENERAL,%d%s,%s %s\n' % (i,l,s,feature_english(c)))
        header = False

AR
AT
Austria
Most important features:
* DAYSSINCELASTTRANS
* TRANSCOUNT
* PRODUCTCOUNT

Scoring 8803 end users, with explanations for the top 1760 scores... AU
Australia
Most important features:
* EmployeesTotalFinal
* TRANSCOUNT
* EXTENDEDSALES

Scoring 208744 end users, with explanations for the top 10000 scores... BE
Belgium
Most important features:
* EmployeesTotalFinal
* TRANSCOUNT
* DAYSSINCELASTTRANS

Scoring 33813 end users, with explanations for the top 6762 scores... BR
Brazil
Most important features:
* DAYSSINCELASTTRANS
* EmployeesTotalFinal
* TRANSCOUNT3M

Scoring 20167 end users, with explanations for the top 4033 scores... CH
Switzerland
Most important features:
* TRANSCOUNT
* EmployeesTotalFinal
* PRODUCTCOUNT

Scoring 34687 end users, with explanations for the top 6937 scores... CL
Chile
Most important features:
* DAYSSINCELASTTRANS
* TRANSCOUNT
* TRANSCOUNT6M

Scoring 4773 end users, with explanations for the top 954 scores... CO
Colombia
Most important features:
* E

In [8]:
pd.read_csv(outfolder + 'TECHINDEX.csv', dtype={'DM_ULT_DUN':'str'}).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3110932 entries, 0 to 3110931
Data columns (total 9 columns):
 #   Column           Dtype  
---  ------           -----  
 0   LEVEL            object 
 1   CALMONTHID       int64  
 2   COMPANYCD        object 
 3   DM_ULT_DUN       object 
 4   GROUP            float64
 5   RAWSCORE         float64
 6   TECHINDEX_VALUE  float64
 7   REASON1CD        object 
 8   REASON2CD        object 
dtypes: float64(3), int64(1), object(5)
memory usage: 213.6+ MB


In [9]:
engine1 = create_engine('mssql://uschwsql1056d/DEV_ANALYTICS?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server',\
                       fast_executemany=True)

pd.read_csv(outfolder + 'TECHINDEX_REASONS.csv').to_sql('TECHINDEX_GENERAL_LOAD_REASONS', con=engine1,
                                                        schema='dbo', if_exists='append',
                                                        chunksize=100000, index=False)
pd.read_csv(outfolder + 'TECHINDEX.csv',
            dtype={'DM_ULT_DUN':'str'}).to_sql('TECHINDEX_GENERAL_LOAD', con=engine1,
                                                        schema='dbo', if_exists='append',
                                                        chunksize=100000, index=False)